# Uso de MLFLOW
En el notebook anterior, hicimos la exploración de los datos de un dataset de reseñas de videojuegos a partir del cual hice la práctica del módulo NLP de KC. 
En este caso, vamos a cargar los modelos ya hechos en este cuaderno para cargar estos modelos en MLFLOW y 

In [3]:
import joblib
import pandas as pd

# rutas del modelo 1
ruta_modelo_1 = "modelos/Logistic_regression_tf_idf0-1/model.pkl"
ruta_vectorizer_1 = "modelos/Logistic_regression_tf_idf0-1/vectorizer.pkl"

# cargamos modelo y vectorizador
modelo_1 = joblib.load(ruta_modelo_1)
vectorizer_1 = joblib.load(ruta_vectorizer_1)

# datos de prueba
data = {'text': ['I love this product!', 'This is the worst service ever.']}
df = pd.DataFrame(data)

# transformación + predicción
X_1 = vectorizer_1.transform(df['text'])
pred_1 = modelo_1.predict(X_1)

print("Predicciones modelo 1:", pred_1)



Predicciones modelo 1: [1 0]


In [4]:
# rutas del modelo 2
ruta_modelo_2 = "modelos/Logistic_regression_tf_idf0-5/model.pkl"
ruta_vectorizer_2 = "modelos/Logistic_regression_tf_idf0-5/vectorizer.pkl"

# cargamos modelo y vectorizador
modelo_2 = joblib.load(ruta_modelo_2)
vectorizer_2 = joblib.load(ruta_vectorizer_2)

# transformación + predicción
X_2 = vectorizer_2.transform(df['text'])
pred_2 = modelo_2.predict(X_2)

print("Predicciones modelo 2:", pred_2)


Predicciones modelo 2: [5 1]


Tras hacer esta primera carga, observamos dos cosas: 
- El modelo debe cargarse junto con el vectorizer
- VScode nos avisa de que el venv tiene versiones de sus librerías distintas a aquellas con las que se creó el modelo. Por ello, vamos a crear un venv en Anaconda con las dependencias exactas para evitar 
- al hacer un predict sobre data, al tener una lista con 2 reseñas artificiales, nos devuelve una lista con las dos predicciones.
        - 'I love this product!' tiene una puntuación 1, la más alta
        - 'This is the worst service ever.' tiene una puntuación 0, la más baja
    - En el modelo 2, el resultado devuelto es [5,1], por lo que:
        - 'I love this product!' tiene una puntuación 5, la más alta
        - 'This is the worst service ever.' tiene una puntuación 1, la más baja

In [11]:
import mlflow
MLFLOW_TRACKING_URI = 'http://localhost:5000'
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [7]:
# para conseguir estadísticas reales de los modelos, traemos el dataset ya preprocesado y lo dividimos 
from sklearn.model_selection import train_test_split
ruta_dataset_1 = "reviews_Video_Games_5_balanced_preprocessed_0-1.csv"
ruta_dataset_2 = "reviews_Video_Games_5_balanced_preprocessed.csv"
df_1 = pd.read_csv(ruta_dataset_1)
df_2 = pd.read_csv(ruta_dataset_2)
X_1_full = vectorizer_1.transform(df_1['reviewText'])
y_1_full = df_1['overall']
X_2_full = vectorizer_2.transform(df_2['reviewText'])
y_2_full = df_2['overall']
# Vamos a dividir en train y test, usando el mismo random state para ambos modelos que usamos al entrenarlos durante la práctica para reproducir los mismos resultados

X_1_train, X_1_test, y_1_train, y_1_test = train_test_split(X_1_full, y_1_full, test_size=0.2, random_state=42)
X_2_train, X_2_test, y_2_train, y_2_test = train_test_split(X_2_full, y_2_full, test_size=0.2, random_state=42)



In [8]:
accuracy_test_1 = modelo_1.score(X_1_test, y_1_test); accuracy_test_1
accuracy_train_1 = modelo_1.score(X_1_train, y_1_train); accuracy_train_1
accuracy_test_2 = modelo_2.score(X_2_test, y_2_test); accuracy_test_2
accuracy_train_2 = modelo_2.score(X_2_train, y_2_train); accuracy_train_2

print(f"Accuracy Test Modelo 1: {accuracy_test_1}, Accuracy Train Modelo 1: {accuracy_train_1}")
print(f"Accuracy Test Modelo 2: {accuracy_test_2}, Accuracy Train Modelo 2: {accuracy_train_2}")

Accuracy Test Modelo 1: 0.79625, Accuracy Train Modelo 1: 0.91
Accuracy Test Modelo 2: 0.416, Accuracy Train Modelo 2: 0.778


In [9]:
import mlflow
import mlflow.sklearn

mlflow.set_experiment("sentiment-analysis-logreg")


2026/01/29 13:07:48 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.schemas
2026/01/29 13:07:48 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.tables
2026/01/29 13:07:48 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.types
2026/01/29 13:07:48 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.constraints
2026/01/29 13:07:48 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.defaults
2026/01/29 13:07:48 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.comments
2026/01/29 13:07:48 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/01/29 13:07:48 INFO alembic.runtime.migration: Will assume non-transactional DDL.


<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1769684646051, experiment_id='1', last_update_time=1769684646051, lifecycle_stage='active', name='sentiment-analysis-logreg', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [15]:
with mlflow.start_run(run_name="LogReg_TFIDF_0.1"):
# Guardamos métricas, parámetros, modelo y vectorizador del modelo 1
    mlflow.log_metric("accuracy_train", accuracy_train_1)
    mlflow.log_metric("accuracy_test", accuracy_test_1)
# Guardamos parámetros
    mlflow.log_param("model_type", "LogisticRegression")
    mlflow.log_param("tfidf_max_df", 0.1)
# Guardamos modelo 
    mlflow.sklearn.log_model(
        sk_model=modelo_1,
        artifact_path="logreg_tfidf_0_1"
    )
# Guardamos vectorizador
    mlflow.log_artifact(
        "modelos/Logistic_regression_tf_idf0-1/vectorizer.pkl",
        artifact_path="vectorizer_tfidf_0_1"
    )

2026/01/29 13:27:49 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run LogReg_TFIDF_0.1 at: http://localhost:5000/#/experiments/1/runs/248f549fbb3748d4911c61b382d295b4
🧪 View experiment at: http://localhost:5000/#/experiments/1


In [ ]:
#Hacemos lo mismo con el segundo modelo
with mlflow.start_run(run_name="LogReg_TFIDF_0.5"):

    # métricas
    mlflow.log_metric("accuracy_train", accuracy_train_2)
    mlflow.log_metric("accuracy_test", accuracy_test_2)

    # parámetros
    mlflow.log_param("model_type", "LogisticRegression")
    mlflow.log_param("tfidf_max_df", 0.5)

    #  modelo
    mlflow.sklearn.log_model(
        sk_model=modelo_2,
        artifact_path="logreg_tfidf_0_5"
    )

    # vectorizer
    mlflow.log_artifact(
        "modelos/Logistic_regression_tf_idf0-5/vectorizer.pkl",
        artifact_path="vectorizer_tfidf_0_5"
    )

2026/01/29 13:27:52 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run LogReg_TFIDF_0.5 at: http://localhost:5000/#/experiments/1/runs/77ba1d07e08e4ba88b35281416b499fa
🧪 View experiment at: http://localhost:5000/#/experiments/1


Tras algunas ejecuciones, he comprobado que todos los modelos estaban siendo guardados bajo el mismo nombre (model), por lo que he cambiado el parámetro necesario (artifact_path) para tener una pantalla de métricas comprensible, más declarativa y clara 